<a href="https://colab.research.google.com/github/kgautam2103/Meetup_automation/blob/main/create_events_meetup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# when you run this code, you will get a link below
#Follow the link, copy the code, paste it in the box and then press Enter on the keyboard.
#This will authenticate you and will allow you to interact with Google Sheets and other Google apps in your notebook.
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
client_key="emgo5alo5672bdgr1k4bp6sn57"

client_redirect_url="https://us.sahajayoga.org/"

authorize_url = "https://secure.meetup.com/oauth2/authorize?client_id="+client_key+"&response_type=code&redirect_uri="+client_redirect_url

print(authorize_url)

https://secure.meetup.com/oauth2/authorize?client_id=emgo5alo5672bdgr1k4bp6sn57&response_type=code&redirect_uri=https://us.sahajayoga.org/


In [ ]:
import requests
import json
import pandas as pd
import time
from datetime import datetime
from dateutil import tz

def get_access_refresh_token_from_code(access_code,client_secret, grant_type_code):
  access_url = "https://secure.meetup.com/oauth2/access?client_id="+client_key+"&client_secret="+client_secret+"&grant_type="+grant_type_code+"&redirect_uri="+client_redirect_url+"&code="+access_code


  response = requests.post(access_url)

  print(response.json())

  access_token = response.json()["access_token"]
  refresh_token = response.json()["refresh_token"]

  return access_token,refresh_token


def get_access_refresh_token_from_refresh(refresh_token,client_secret, grant_type_refresh):
  access_url = "https://secure.meetup.com/oauth2/access?client_id="+client_key+"&client_secret="+client_secret+"&grant_type="+grant_type_refresh+"&refresh_token="+refresh_token


  response = requests.post(access_url)

  print(response.json())

  access_token = response.json()["access_token"]
  refresh_token = response.json()["refresh_token"]

  return access_token,refresh_token


access_code = input("please enter the access code recieved ")

client_secret="10dk05f2c9jfu5puidnetpedcp"

grant_type_code = "authorization_code"

grant_type_refresh = "refresh_token"

meetup_api_url = "https://api.meetup.com/gql"

#sheet_name = input('enter the start time in UTC in format 2022-12-31T13:45 : ')

access_token,refresh_token = get_access_refresh_token_from_code(access_code,client_secret, grant_type_code)

please enter the access code recieved 31effe67af37533bf30c0be2922799ab
{'access_token': '00cb68259d9dcb0958ac3300f0d0c65a', 'refresh_token': '8c9e52612e3229225523159c6a2ad142', 'token_type': 'bearer', 'expires_in': 3600}


In [ ]:
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1eXQ6T_67xmq3oH4tqoV5_Lvv6YF0apWgd3xJUv0y1No/edit#gid=0')

current_ts = datetime.now().strftime('%Y%m%d%H%M%S')
output_sheet = current_ts+'_event_output_meetup'

counter=1

# create and Open our new sheet and add some data.
sh = gc.create(output_sheet)
output_sheet = gc.open(output_sheet).sheet1

output_sheet.insert_row(['group_url','group_name','event_id','event_url','timezone','group_id','photo_id','error'],1)

sheet_name = input('enter the name of input sheet : ')
input_sheet = wb.worksheet(sheet_name)
inputdata = input_sheet.get_all_values()
inputdata_df = pd.DataFrame(inputdata, columns=['group_name',	'sample_event_id','photo_id'])
inputdata_df = inputdata_df.drop(inputdata_df.index[0])


for index,row in inputdata_df.iterrows():
  counter = counter+1
  created_event_id=""
  created_event_url=""
  created_event_error=""
  access_token,refresh_token = get_access_refresh_token_from_refresh(refresh_token,client_secret, grant_type_refresh)
  authorization = 'Bearer ' + access_token
  auth_headers = {
    'Authorization': authorization,
    'Content-Type': 'application/json'
    }

  print("access_token "+access_token)
  print("refresh_token "+refresh_token)
  cell_value_list = []
  photo_id = int(row['photo_id'])
  event_id = row['sample_event_id']
  group_url = str(row['group_name'])

  group_name = group_url.split("/")[3]

  print(photo_id, event_id, group_name)

  group_query = """query($grp_url : String!){
              groupByUrlname(urlname: $grp_url){
              id
              timezone
              }
            }"""

  variables_group_query = {"grp_url":group_name}

  gql_group_query_response = requests.post(meetup_api_url,json={'query': group_query, 'variables': variables_group_query}, headers=auth_headers)

  print(gql_group_query_response.text)

  group_id = gql_group_query_response.json()['data']['groupByUrlname']['id']
  print(group_id)
  grp_timezone = gql_group_query_response.json()['data']['groupByUrlname']['timezone']
  print(grp_timezone)

  event_query = """query ($event_id : ID) {
    event(id: $event_id) {

      id
      image {id, baseUrl}
      venue {id, lat, lng}
      hosts {id}
      priceTier
      timezone
      dateTime
      duration
      endTime
      title
      description
    }
  }"""


  variables_event_query = {"event_id":event_id}

  gql_event_query_response = requests.post(meetup_api_url,json={'query': event_query, 'variables': variables_event_query}, headers=auth_headers)

  gql_event_query_response_json = json.loads(gql_event_query_response.text)
  print(gql_event_query_response.json())

  title = gql_event_query_response_json['data']['event']['title']
  description = gql_event_query_response_json['data']['event']['description']
  start_date_time = gql_event_query_response_json['data']['event']['dateTime']
  venue_id = gql_event_query_response_json['data']['event']['venue']['id']
  duration = gql_event_query_response_json['data']['event']['duration']
  event_timezone = gql_event_query_response_json['data']['event']['timezone']
  image_id = int(gql_event_query_response_json['data']['event']['image']['id'])
  image_base_url = gql_event_query_response_json['data']['event']['image']['baseUrl']
  priceTier = gql_event_query_response_json['data']['event']['priceTier']
  hosts_list = gql_event_query_response_json['data']['event']['hosts']

  host_id_list=[]
  for x in hosts_list:
    print(x['id'])
    host_id_list.append(int(x['id']))

  print(host_id_list)

  from_zone = tz.gettz(event_timezone)
  to_zone = tz.gettz(grp_timezone)

  start_date_time1= start_date_time.replace("Z",'')
  if len(start_date_time1)>16:
    start_date = start_date_time1[:-6]
  else:
    start_date = start_date_time1

  start_timestamp = datetime.strptime(start_date,"%Y-%m-%dT%H:%M")
  print(start_timestamp)
  old_timezone = start_timestamp.replace(tzinfo=from_zone)
  new_timezone = old_timezone.astimezone(to_zone)
  print(old_timezone)
  print(new_timezone)
  date_input = datetime.strftime(new_timezone,"%Y-%m-%dT%H:%M")
  print(date_input)

  create_event_query = """
      mutation($create_event_input: CreateEventInput!) {
    createEvent(input: $create_event_input) {
      event {
        id
      }
      errors {
        message
        code
        field
      }
    }
  }
  """


  create_event_variables = {
    "create_event_input": {
      "groupUrlname": group_name,
      "title": title,
      "description": description,
      "startDateTime": date_input,
      "venueId": "online",
      "duration": duration,
      "featuredPhotoId":photo_id,
      "venueVisibility":"PUBLIC",
      "howToFindUs":"https://zoom.us/j/9181716151",
      "eventHosts":host_id_list
    }
  }

  create_event_mutation_response = requests.post(meetup_api_url, json={'query': create_event_query, 'variables': create_event_variables}, headers=auth_headers)

  print(create_event_mutation_response.text)

  try:
    created_event_error=create_event_mutation_response.json()['data']['createEvent']['errors'][0]['message']
    created_event_id=""
    created_event_url=""
  except:
    created_event_error=""
    created_event_id=create_event_mutation_response.json()['data']['createEvent']['event']['id']
    created_event_url=group_url+"events/"+created_event_id+"/"

  cell_value_list = [group_url,group_name,created_event_id,created_event_url,grp_timezone,group_id,photo_id,created_event_error]
  print(cell_value_list)
  output_sheet.insert_row(cell_value_list,counter)

{'access_token': 'c9d1fcc816270b165495f1be8e8dc90b', 'refresh_token': '8c9e52612e3229225523159c6a2ad142', 'token_type': 'bearer', 'expires_in': 3600}
access_token c9d1fcc816270b165495f1be8e8dc90b
refresh_token 8c9e52612e3229225523159c6a2ad142
509054507 298395203 Anchorage-Sahaja-Yoga-Meditation
{"data":{"groupByUrlname":{"id":"31546705","timezone":"America/Anchorage"}}}

31546705
America/Anchorage
{'data': {'event': {'id': '298395203', 'image': {'id': '518355800', 'baseUrl': 'https://secure-content.meetupstatic.com/images/classic-events/'}, 'venue': {'id': '26906060', 'lat': -8.521147, 'lng': 179.1962}, 'hosts': [{'id': '14390233'}], 'priceTier': 'paid', 'timezone': 'America/New_York', 'dateTime': '2024-01-27T20:00-05:00', 'duration': 'PT1H', 'endTime': '2024-01-27T21:00-05:00', 'title': 'Curso de Meditación de 21 Días En Línea Gratis en Español', 'description': '**¡Prepárate para una experiencia transformadora que te cambiará la vida!**\n\n**Sábado 27 de enero – viernes 16 de febrero 